This code is meant to run mianalyzer headless. Mianalyzer is a code that can run a variety of neural network architrecture and backbones, without the user having ot define them himeself. It normally runs as a GUI application. However, when run on a remote server, the GUI version is much too slow,  The mianalyzer author (koerber) kindly provided a python script to run the training sequence  headless, after loading a "pickled"dl object from a pkl file. 

Inputs:
    1. Directory that holds the pkl file
    2. Directory where the model will be saved
    3. The name of the saved pkl file (the pkl extensionis assumed)
    4. The directory containing the training data. The Segmentation_labels subdiretory must also be present.
    5. The prediction folder containing images on which the trained model will be tested. 
    6. model backbone (default is densenet201)
    7. number of epochs (default defined in the pkl file)

 Outputs:
    1. The dl object that will be used to train the model
    2. The trained model (a .h5 file, and a .csv file with the loss history)
    3. A subdirectory that will hold the segmented images.

This Jupyter script is based on code provided by mianalzer author Nils Koerber (nils.koerber@bfr.bund.de)

Modifed by Aryeh Weiss
Last modified: 19 June 2023

This version of the script usees tkinter to prompt for teh various directories and files that are needed.
The defaults are the same, and cliking ok will select default values.
When run on a remote GPU server, this version will be slower because it opens X windows on the local machine,
pver the network.

TODO: 1. Add a drop down menu for all of the possible backbones.
      2. Change default backone to be that of teh pkl file (currenlty it is densenet201)

In [1]:
import pickle
import os 
import cv2
import glob
import matplotlib.pyplot as plt

from pprint import pprint

import tkinter as tk
from tkinter import *
from tkinter import filedialog as fd


# select one of the multiple GPU cards on the server.
gpuNumber = input("Select GPU or enter for 0\n") or '0'

os.environ["CUDA_VISIBLE_DEVICES"]=gpuNumber  # set GPU if multiple present

os.environ['DISPLAY']='localhost:10.0'


In [3]:
'''
This code is developed and run on at least two machines
Here we setup the default paths for each machine.
Eventually we must find a more general way to do this.
'''

host = os.getenv('HOSTNAME')
if host==None:      # pop os dees not have HOSTNAME defined by default.
    host = "pop"
print(host)

if "dsigpu" in host:
    pklDir = '/home/dsi/aryeh/data/mia/pkl/'
    modelDir = '/home/dsi/aryeh/data/mia/trained_models/'
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata = '/home/dsi/aryeh/data/plants/unCropped/resized/'
    predictionFolder = '/home/dsi/aryeh/data/plants/Harvest8Orange5_7Oct17/'
elif "pop" in host:
    pklDir = '/media/amw/TOSHIBA EXT/alerding/models/pkl/'
    modelDir = '/media/amw/TOSHIBA EXT/alerding/models/gpuModels/'
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata =  '/media/amw/TOSHIBA EXT/alerding/annotated/notCroppedLabels2/resized/'
    predictionFolder = '/media/amw/TOSHIBA EXT/alerding/Harvest 8 Orange 5,7 Oct 17.r/vertical/'
else:
    pklDir = None
    modelDir = None 
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata = None
    predictionFolder = None

dsigpu02


In [4]:
'''
Put this in a separate cell so that multiple instances of Tk are not invoked
'''
root = Tk()
root.withdraw()

''

In [5]:

pklDir = fd.askdirectory(initialdir=pklDir) + os.sep
modelDir = fd.askdirectory(initialdir=modelDir) + os.sep
trainingdata = fd.askdirectory(initialdir=trainingdata) + os.sep
predictionFolder = fd.askdirectory(initialdir=predictionFolder) + os.sep



modelFilePath = fd.askopenfilename(
        title='Open a pkl file',
        initialdir=pklDir)

filename = os.path.basename(modelFilePath)
modelname = os.path.splitext(filename)[0]

print("model: ", modelname)

validationdata = None

loadweights = None



model:  unetDensenet201ep75_221108


In [6]:

#Set the window geometry
root.geometry("200x200")
root.deiconify()

# Create object
# root = Tk()

# Adjust size
# root.geometry( "200x200" )

# Change the label text
def show():
	label.config( text = clicked.get() )

# list of backbones
backbones = [None, 'resnet18', 'resnet34','resnet50', 'resnet101', 'resnet152',
             'seresnet18', 'seresnet34', 'seresnet50', 'seresnet101', 'seresnet152', 'seresnext101',
             'senet154','resnext50', 'resnext101', 'vgg16', 'vgg19',
             'densenet121', 'densenet169', 'densenet201', 'inceptionresnetv2', 'inceptionv3',
             'mobilenet', 'mobilenetv2',
             'efficientnetb0','efficientnetb1','efficientnetb02','efficientnetb3','efficientnetb4','efficientnetb5','efficientnetb6','efficientnetb7']

# datatype of menu text
clicked = StringVar()

# Create Dropdown menu
drop = OptionMenu( root , clicked , *backbones )
drop.pack()

# Create Label
label = Label( root , text = "Choose backbone" )
label.pack()

Button(root, text="Quit", command=root.destroy).pack() 
  
# Execute tkinter
root.mainloop()

BackBone = clicked.get()
print(BackBone)

strEpochs = input("input number of epochs or enter to use the predefined value in a pkl file\n") or '0'
epochs = int(strEpochs)

mobilenetv2


In [7]:
'''
This is from the version that reads the dl object from a pkl file.
'''

print('load settings')
#filehandler = open(modelname + '.pkl', 'rb')
filehandler = open(pklDir + modelname + '.pkl', 'rb')

dl = pickle.load(filehandler)
print(dl.epochs)
if epochs != 0:
    dl.epochs = epochs
if BackBone != None:
    dl.Mode.backbone = BackBone
pprint(vars(dl.Mode))


load settings


2023-07-05 17:44:51.382892: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


75
{'architecture': 'UNet',
 'backbone': 'mobilenetv2',
 'loss': <dl.loss.segmentation_losses.SegmentationLosses object at 0x7fbf80dc0f10>,
 'metric': <dl.metric.segmentation_metrics.SegmentationMetrics object at 0x7fbf80dc0d10>,
 'parent': <dl.DeepLearning.DeepLearning object at 0x7fbfcc0b9a50>,
 'preprocessingfnc': None,
 'pretrained': True,
 'type': <dlMode.Segmentation: 2>}


In [8]:
'''
Run this cell if you want to save the dl object as a pkl file
'''

pklFileHandler = open(pklDir+dl.Mode.architecture+'_'+dl.Mode.backbone+'_ep'+str(dl.epochs) + '.pkl','wb')
#import sys
#print(sys.getrecursionlimit())
#sys.setrecursionlimit(4*sys.getrecursionlimit())
#print(sys.getrecursionlimit())

hed = dl.hed
dl.hed = None
model = dl.Model
dl.Model = None
pickle.dump(dl, pklFileHandler)
dl.hed = hed
dl.Model = model



In [9]:
# pod, stem, background, and unlabeled (for deleaved soybean plants)
numClasses =4

print('init model')
dl.initModel(numClasses)

dl.Model = dl.Mode.getModel(numClasses, 3)

print('load model')
if loadweights is not None:
    dl.Model.load_weights(loadweights)

print(dl.initialized)

init model


/home/dsi/aryeh/git/miaHeadless/mia/dl/models/keras_applications/mobilenet_v2.py:296: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
2023-07-05 17:46:35.100066: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-05 17:46:35.116908: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-05 17:46:35.161204: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.62GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2023-07-05 17:46:35.161259: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudar

load model
True


In [10]:
'''
Here we  do the training
'''
        
if dl.initialized:
    print('start training')
    dl.initData_StartTraining(trainingdata, validationdata)
    print('training finished')
else:
    print('could not initialize model')
            
print('saving weights')
modelname=dl.Mode.architecture+'_'+dl.Mode.backbone
modelPath = modelDir+modelname+'_ep'+str(dl.epochs) + '.h5'
print('model path: ', modelPath)
dl.Model.save_weights(modelPath)
print('saving training data')
dl.saveTrainingRecord(modelDir+modelname+'_ep'+str(dl.epochs) +'.csv')
#   print('saving model')
#   dl.Model.save(modelDir+'full')
    



start training


2023-07-05 17:47:07.829014: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-07-05 17:47:07.830182: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199850000 Hz


Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-07-05 17:47:15.669780: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-07-05 17:47:17.212518: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


133/133 [==============================] - 31s 159ms/step - loss: 0.0069 - pixel_accuracy: 194045.5169 - val_loss: 0.0621 - val_pixel_accuracy: 9219.3252
Epoch 2/50
133/133 [==============================] - 17s 126ms/step - loss: 0.0054 - pixel_accuracy: 203210.4762 - val_loss: 0.0502 - val_pixel_accuracy: 54875.4492
Epoch 3/50
133/133 [==============================] - 17s 124ms/step - loss: 0.0056 - pixel_accuracy: 220146.6813 - val_loss: 0.0220 - val_pixel_accuracy: 16626.6250
Epoch 4/50
133/133 [==============================] - 16s 112ms/step - loss: 0.0050 - pixel_accuracy: 210103.8587 - val_loss: 0.0132 - val_pixel_accuracy: 74128.5234
Epoch 5/50
133/133 [==============================] - 16s 115ms/step - loss: 0.0046 - pixel_accuracy: 227144.1705 - val_loss: 0.0189 - val_pixel_accuracy: 9831.0000
Epoch 6/50
133/133 [==============================] - 15s 109ms/step - loss: 0.0048 - pixel_accuracy: 225589.8122 - val_loss: 0.0255 - val_pixel_accuracy: 7896.6499
Epoch 7/50
133/133

In [ ]:
'''
This cell does a prediction on a directory of jpg images (could be any type, but that is what I have).
predictionFolder contains the images which ill be predicted, and a subdirectory with the modelname will be created
to hold the predicted segmentation.

A h5 file with pretrained weights can be loaded, in which case the traiing step above can be skipped.
However, the previous cells tha set up the dl object must be run. I have not yet succeeded in defining 
a properly working dl object wihtout first usin a saved pkl file.

To use the model that was just trained, press "cancel" when prompted for a weights file.

'''


modelWeightsPath = fd.askopenfilename(
        title='Open a weights file (h5)',
        initialdir=modelDir)

print(modelWeightsPath)

if modelWeightsPath != "":
    dl.Model.load_weights(modelWeightsPath)

predictionFolder = fd.askdirectory(initialdir=predictionFolder) + os.sep
outputPath = predictionFolder + modelname+'_ep'+str(dl.epochs)+'/'
try:
    os.mkdir(outputPath)
except FileExistsError:
    pass
files = glob.glob(predictionFolder + '*.jpg')
print(len(files))



# If we do not want to process the entire folder, we can limit the number of processed image to count. 
# If the entire folder is processed, then it wold be agood idea to comment out the image display 
count = 0
for i in files:
    print(i)
    img = cv2.imread(i)
    prediction = dl.Mode.PredictImage(img)
    plt.figure(count)
    plt.imshow(prediction)
    plt.show()
    cv2.imwrite(outputPath+'segmented_'+os.path.basename(i).replace('jpg', 'png'), prediction)
    count += 1
#    if count > 10:
#       break
    



In [ ]:
print(predictionFolder)